# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import sklearn
import xgboost as xgb
import lightgbm as lgb

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from yellowbrick.cluster import KElbowVisualizer
from scipy.cluster.hierarchy import dendrogram, linkage, leaves_list
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn. ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from plotly.subplots import make_subplots

# Read Data

In [ ]:
df = pd.read_csv('Chicago-Dataset.csv')

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df.head()

In [ ]:
df.tail()

# Data Discovery

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

# Data Correlation

In [ ]:
# Heatmap
df['sold_price'] = pd.to_numeric(df['sold_price'], errors='coerce')
df_numeric = df.select_dtypes(include='number')

correlation = (
    df_numeric
    .corr()['sold_price']
    .dropna()
    .sort_values(ascending=False)
)

correlation = correlation[correlation.index != 'sold_price']
correlation_df = correlation.to_frame(name='Correlation').round(3)

fig_heatmap = px.imshow(
    correlation_df,
    text_auto=True,
    aspect="auto",
    color_continuous_scale="RdBu",
    zmin=-1,
    zmax=1,
    title="Correlation Heatmap: Features vs Sold Price"
)

fig_heatmap.update_layout(
    autosize=True,
    height=600,
    margin=dict(l=60, r=60, t=80, b=60),
    font=dict(size=14)
)

fig_heatmap.show()

In [ ]:
# Bar chart
fig_bar = px.bar(
    correlation_df,
    x='Correlation',
    y=correlation_df.index,
    orientation='h',
    title="Feature Correlation Strength with Sold Price",
    text='Correlation'
)

fig_bar.update_layout(
    autosize=True,
    height=700,
    margin=dict(l=120, r=60, t=80, b=60),
    font=dict(size=14)
)

fig_bar.update_traces(textposition='outside')
fig_bar.show()

In [ ]:
# Scatter plot
top_feature = correlation_df.index[0]

fig_scatter = px.scatter(
    df,
    x=top_feature,
    y='sold_price',
    trendline='ols',
    title=f"Sold Price vs {top_feature}",
    opacity=0.7
)

fig_scatter.update_layout(
    autosize=True,
    height=600,
    margin=dict(l=60, r=60, t=80, b=60),
    font=dict(size=14)
)

fig_scatter.show()

# Column Cleaning

In [ ]:
df_columns_drop = df.copy()

In [ ]:
print(df.columns)

In [ ]:
columns_drop = ['id', 'property_id', 'listing_number', 'status_changed_on', 'created_at', 'updated_at', 'status', 'contracted_on', 'off_market_on', 'original_list_price', 'previous_price', 'seller_concessions', 'finished_square_feet', 'derived_basement_square_feet', 'car_storage', 'car_spaces', 'area', 'subdivision', 'street', 'state', 'county', 'property_key', 'externally_last_updated_at', 'photos',  'photos_pulled', 'structural_style', 'architecture', 'lot_size_square_feet', 'lot_size_acres', 'basement_finished_status', 'basement_finished_pct', 'basement_square_feet', 'basement_size', 'basement_type', 'seller_type', 'public_remarks', 'description', 'lat', 'lng', 'zoned', 'showings_phone', 'approval_condition', 'listing_agent', 'listing_brokerage', 'is_attached', 'stories', 'version', 'parcel_number', 'hoa_name', 'school_district', 'private_remarks', 'price_changes', 'unit_count', 'county_data_id']

In [ ]:
df_columns_drop = df_columns_drop.drop(columns = columns_drop, axis = 1)

In [ ]:
print(df_columns_drop.columns)

# Preprocessing

In [ ]:
df.info()

In [ ]:
# Convert zip to number
df_columns_drop['zip'] = pd.to_numeric(df_columns_drop['zip'], errors='coerce')

In [ ]:
# Calculate the average days on market
temp_df = df_columns_drop[['listed_on', 'sold_on']].copy()

temp_df['listed_on'] = pd.to_datetime(temp_df['listed_on'], errors='coerce')
temp_df['sold_on'] = pd.to_datetime(temp_df['sold_on'], errors='coerce')

temp_df = temp_df.dropna(subset=['listed_on', 'sold_on'])

temp_df['days_on_market'] = (temp_df['sold_on'] - temp_df['listed_on']).dt.days

temp_df = temp_df[temp_df['days_on_market'] >= 0]

average_days_on_market = temp_df['days_on_market'].mean()
average_days_on_market

# Data Cleaning

In [ ]:
# Sort the missing value by ascending order
df_columns_drop.isnull().sum().sort_values()

In [ ]:
# Drop if a row misses more than 10 values
drop_threshold = 10
df_columns_drop = df_columns_drop[df_columns_drop.isnull().sum(axis=1) <= drop_threshold]

In [ ]:
# Start cleaning by this order
df_columns_drop.isnull().sum().sort_values()

In [ ]:
# Clean the zip by finding the mode of the same city
zip_mode_by_city = (
    df_columns_drop
    .dropna(subset=['zip', 'city'])
    .groupby('city')['zip']
    .agg(lambda x: x.mode().iloc[0])
)

df_columns_drop['zip'] = df_columns_drop['zip'].fillna(
    df_columns_drop['city'].map(zip_mode_by_city)
)

df_columns_drop = df_columns_drop.dropna(subset=['zip']).drop(columns=['city'])

In [ ]:
# Clean listed on by subtracting the sold_on with average market days
df_columns_drop['sold_on'] = pd.to_datetime(df_columns_drop['sold_on'], errors='coerce')
df_columns_drop['listed_on'] = pd.to_datetime(df_columns_drop['listed_on'], errors='coerce')

missing_listed_mask = df_columns_drop['listed_on'].isna() & df_columns_drop['sold_on'].notna()
df_columns_drop.loc[missing_listed_mask, 'listed_on'] = (
    df_columns_drop.loc[missing_listed_mask, 'sold_on'] - pd.to_timedelta(average_days_on_market, unit='d')
)

df_columns_drop = df_columns_drop.dropna(subset=['listed_on'])

In [ ]:
# Ensure numeric values
df_columns_drop['baths'] = pd.to_numeric(df_columns_drop['baths'], errors='coerce')
df_columns_drop['above_grade_square_feet'] = pd.to_numeric(df_columns_drop['above_grade_square_feet'], errors='coerce')

# Fill by median of baths from similar houses (same range of square feet)
size_tolerance = 200
missing_baths_mask = df_columns_drop['baths'].isna() & df_columns_drop['above_grade_square_feet'].notna()

for idx in df_columns_drop[missing_baths_mask].index:
    target_sqft = df_columns_drop.loc[idx, 'above_grade_square_feet']
    similar_houses = df_columns_drop[
        (df_columns_drop['above_grade_square_feet'] >= target_sqft - size_tolerance) &
        (df_columns_drop['above_grade_square_feet'] <= target_sqft + size_tolerance) &
        (df_columns_drop['baths'].notna())
    ]
    if not similar_houses.empty:
        df_columns_drop.loc[idx, 'baths'] = similar_houses['baths'].median()

# Fill by the median from the same zip
missing_baths_mask = df_columns_drop['baths'].isna() & df_columns_drop['zip'].notna()
baths_median_by_zip = df_columns_drop.groupby('zip')['baths'].median()
df_columns_drop.loc[missing_baths_mask, 'baths'] = df_columns_drop.loc[missing_baths_mask, 'zip'].map(baths_median_by_zip)

# Drop if none method works
df_columns_drop = df_columns_drop.dropna(subset=['baths'])

In [ ]:
# Apply the same logic for beds
df_columns_drop['beds'] = pd.to_numeric(df_columns_drop['beds'], errors='coerce')

size_tolerance = 200
missing_beds_mask = df_columns_drop['beds'].isna() & df_columns_drop['above_grade_square_feet'].notna()

for idx in df_columns_drop[missing_beds_mask].index:
    target_sqft = df_columns_drop.loc[idx, 'above_grade_square_feet']
    similar_houses = df_columns_drop[
        (df_columns_drop['above_grade_square_feet'] >= target_sqft - size_tolerance) &
        (df_columns_drop['above_grade_square_feet'] <= target_sqft + size_tolerance) &
        (df_columns_drop['beds'].notna())
    ]
    if not similar_houses.empty:
        df_columns_drop.loc[idx, 'beds'] = similar_houses['beds'].median()

missing_beds_mask = df_columns_drop['beds'].isna() & df_columns_drop['zip'].notna()
beds_median_by_zip = df_columns_drop.groupby('zip')['beds'].median()
df_columns_drop.loc[missing_beds_mask, 'beds'] = df_columns_drop.loc[missing_beds_mask, 'zip'].map(beds_median_by_zip)

df_columns_drop = df_columns_drop.dropna(subset=['beds'])

In [ ]:
# Clean year_built by median from the same zip
df_columns_drop['year_built'] = pd.to_numeric(df_columns_drop['year_built'], errors='coerce')

year_built_median_by_zip = df_columns_drop.groupby('zip')['year_built'].median()

missing_year_mask = df_columns_drop['year_built'].isna() & df_columns_drop['zip'].notna()
df_columns_drop.loc[missing_year_mask, 'year_built'] = df_columns_drop.loc[missing_year_mask, 'zip'].map(year_built_median_by_zip)

df_columns_drop = df_columns_drop.dropna(subset=['year_built'])

In [ ]:
# Same logic apply to clean total_square_feet
df_columns_drop['total_square_feet'] = pd.to_numeric(df_columns_drop['total_square_feet'], errors='coerce')

total_sqft_median_by_zip = df_columns_drop.groupby('zip')['total_square_feet'].median()

missing_sqft_mask = df_columns_drop['total_square_feet'].isna() & df_columns_drop['zip'].notna()
df_columns_drop.loc[missing_sqft_mask, 'total_square_feet'] = df_columns_drop.loc[missing_sqft_mask, 'zip'].map(total_sqft_median_by_zip)

df_columns_drop = df_columns_drop.dropna(subset=['total_square_feet'])

In [ ]:
# Then again for above_grade_square_feet
df_columns_drop['above_grade_square_feet'] = pd.to_numeric(df_columns_drop['above_grade_square_feet'], errors='coerce')

above_grade_sqft_median_by_zip = df_columns_drop.groupby('zip')['above_grade_square_feet'].median()

missing_above_sqft_mask = df_columns_drop['above_grade_square_feet'].isna() & df_columns_drop['zip'].notna()
df_columns_drop.loc[missing_above_sqft_mask, 'above_grade_square_feet'] = df_columns_drop.loc[missing_above_sqft_mask, 'zip'].map(above_grade_sqft_median_by_zip)

df_columns_drop = df_columns_drop.dropna(subset=['above_grade_square_feet'])

In [ ]:
# Clean garages by calculating the median of houses with similar total_square_feet
df_columns_drop['garages'] = pd.to_numeric(df_columns_drop['garages'], errors='coerce')
df_columns_drop['total_square_feet'] = pd.to_numeric(df_columns_drop['total_square_feet'], errors='coerce')

size_tolerance = 200
min_sqft = df_columns_drop['total_square_feet'].min()
max_sqft = df_columns_drop['total_square_feet'].max()
bins = range(int(min_sqft), int(max_sqft) + size_tolerance, size_tolerance)
df_columns_drop['sqft_bin'] = pd.cut(df_columns_drop['total_square_feet'], bins)

median_garages_by_bin = df_columns_drop.groupby('sqft_bin')['garages'].median()

missing_garages_mask = df_columns_drop['garages'].isna() & df_columns_drop['sqft_bin'].notna()
df_columns_drop.loc[missing_garages_mask, 'garages'] = df_columns_drop.loc[missing_garages_mask, 'sqft_bin'].map(median_garages_by_bin)

df_columns_drop = df_columns_drop.dropna(subset=['garages'])

df_columns_drop = df_columns_drop.drop(columns=['sqft_bin'])

In [ ]:
# Clean hoa_fee from the same zip
df_columns_drop['hoa_fee'] = pd.to_numeric(df_columns_drop['hoa_fee'], errors='coerce')

hoa_fee_median_by_zip = df_columns_drop.groupby('zip')['hoa_fee'].median()

missing_hoa_mask = df_columns_drop['hoa_fee'].isna() & df_columns_drop['zip'].notna()
df_columns_drop.loc[missing_hoa_mask, 'hoa_fee'] = df_columns_drop.loc[missing_hoa_mask, 'zip'].map(hoa_fee_median_by_zip)

df_columns_drop = df_columns_drop.dropna(subset=['hoa_fee'])

In [ ]:
# Clean sold_on by add the average days to listed_on
df_columns_drop['listed_on'] = pd.to_datetime(df_columns_drop['listed_on'], errors='coerce')
df_columns_drop['sold_on'] = pd.to_datetime(df_columns_drop['sold_on'], errors='coerce')

missing_sold_mask = df_columns_drop['sold_on'].isna() & df_columns_drop['listed_on'].notna()
df_columns_drop.loc[missing_sold_mask, 'sold_on'] = (
    df_columns_drop.loc[missing_sold_mask, 'listed_on'] + pd.to_timedelta(average_days_on_market, unit='d')
)

df_columns_drop = df_columns_drop.dropna(subset=['sold_on'])

In [ ]:
# Fill sold_price by the list_price, then from the same zip
df_columns_drop['sold_price'] = pd.to_numeric(df_columns_drop['sold_price'], errors='coerce')
df_columns_drop['list_price'] = pd.to_numeric(df_columns_drop['list_price'], errors='coerce')

missing_sold_price_mask = df_columns_drop['sold_price'].isna() & df_columns_drop['list_price'].notna()
df_columns_drop.loc[missing_sold_price_mask, 'sold_price'] = df_columns_drop.loc[missing_sold_price_mask, 'list_price']

missing_sold_price_mask = df_columns_drop['sold_price'].isna() & df_columns_drop['zip'].notna()
sold_price_median_by_zip = df_columns_drop.groupby('zip')['sold_price'].median()
df_columns_drop.loc[missing_sold_price_mask, 'sold_price'] = df_columns_drop.loc[missing_sold_price_mask, 'zip'].map(sold_price_median_by_zip)

df_columns_drop = df_columns_drop.dropna(subset=['sold_price'])

df_columns_drop = df_columns_drop.drop(columns=['list_price'])

In [ ]:
# Check if everything is clean
df_columns_drop.isnull().sum().sort_values()

# Column Adding

In [ ]:
# Extracting the year to new columns
df_columns_drop['sold_on_year'] = pd.to_datetime(df_columns_drop['sold_on']).dt.year

In [ ]:
# Creating age column
df_columns_drop["age_of_building"] = df_columns_drop['sold_on_year'] - df_columns_drop['year_built']

In [ ]:
# Drop after done
df_columns_drop = df_columns_drop.drop(columns=['sold_on_year'])
df_columns_drop = df_columns_drop.drop(columns=['year_built'])

In [ ]:
# Count the number of negative values
negative = (df_columns_drop['age_of_building'] < 0).sum()
print(f'Negative: {negative}')

In [ ]:
# Keep only the postive
df_columns_drop = df_columns_drop[df_columns_drop['age_of_building'] >= 0]

In [ ]:
# Convert date columns
df_columns_drop['listed_on'] = pd.to_datetime(df_columns_drop['listed_on'])
df_columns_drop['sold_on'] = pd.to_datetime(df_columns_drop['sold_on'])

# Swap dates if listed_on > sold_on
mask = df_columns_drop['listed_on'] > df_columns_drop['sold_on']
df_columns_drop.loc[mask, ['listed_on', 'sold_on']] = df_columns_drop.loc[mask, ['sold_on', 'listed_on']].values

# Create days_on_market column
df_columns_drop['days_on_market'] = (df_columns_drop['sold_on'] - df_columns_drop['listed_on']).dt.days

In [ ]:
# Drop after done
df_columns_drop = df_columns_drop.drop(columns=['sold_on'])
df_columns_drop = df_columns_drop.drop(columns=['listed_on'])

In [ ]:
negative = (df_columns_drop['age_of_building'] < 0).sum()
print(f'Negative: {negative}')

In [ ]:
df_columns_drop.isnull().sum()

# Dummy

In [ ]:
# Copy before dummy
df_visualization = df_columns_drop.copy(deep = True)

In [ ]:
df_columns_drop.info()

In [ ]:
df_columns_drop = pd.get_dummies(df_columns_drop, columns =['property_type'], drop_first = True)

In [ ]:
df_columns_drop = pd.get_dummies(df_columns_drop, columns =['structural_type'], drop_first = True)

In [ ]:
df_columns_drop.info()

In [ ]:
df_columns_drop.head()

In [ ]:
df_columns_drop.tail()

# Machine Learning

In [ ]:
df_machine_learning = df_columns_drop.copy()

In [ ]:
# Calculate IQR for sold_price
Q1 = df_columns_drop['sold_price'].quantile(0.25)
Q3 = df_columns_drop['sold_price'].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
df_columns_drop = df_columns_drop[
    (df_columns_drop['sold_price'] >= lower_bound) &
    (df_columns_drop['sold_price'] <= upper_bound)
]

In [ ]:
features = [
    'above_grade_square_feet',
    'total_square_feet',
    'garages',
    'beds',
    'baths',
    'zip',
    'hoa_fee',
    'age_of_building',
    'days_on_market',
    'property_type_Detached Single',
    'property_type_Mobile Home',
    'property_type_Multi-family',
    'property_type_Two to Four Units',
    'structural_type_Condo',
    'structural_type_Detached'
]

target = ['sold_price']

# Linear Regression

In [ ]:
X = df_machine_learning[features]
y = df_machine_learning[target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse:.2f}")

# Decision Tree

In [ ]:
X = df_machine_learning[features]
y = df_machine_learning[target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse:.2f}")

# Random Forest

In [ ]:
# Choose either StandardScaler() or MinMaxScaler()
scaler = StandardScaler()

In [ ]:
# Fit the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Parameters
rf_regressor = RandomForestRegressor (
    # Number of trees
    n_estimators = 100,
    # Depth of each tree
    max_depth = 50,
    # Minimum required sample
    min_samples_split = 15,
    # Minimum to be a leaf
    min_samples_leaf = 2,
    # Considered for splitting
    max_features = 0.8,
    # Apply randomness
    random_state = 42
)

In [ ]:
# Train the model
rf_regressor.fit(X_train_scaled, y_train)

In [ ]:
# Perform predictions
rf_predict = rf_regressor.predict(X_test_scaled)

In [ ]:
# Perform calculations
mse = mean_squared_error(y_test, rf_predict)
print(f'MSE: {mse}')

rmse = mean_squared_error(y_test, rf_predict, squared=False)
print(f'RMSE: {rmse}')

r2 = r2_score(y_test, rf_predict)
print(f'R^2 Score: {r2}')

# XGBoost

In [ ]:
# DMatrix fpr train and test
dtrain = xgb. DMatrix(X_train, label=y_train)
dtest = xgb. DMatrix(X_test)

In [ ]:
# Set up parameters
params = {
    # Maximum depth of each decision tree
    'max_depth': 16,
    # Learning rate
    'eta': 0.08,
    # Loss function
    'objective': 'reg:squarederror',
    # Number of parallel threads
    'nthread': 4,
    # Number of estimators
    'n_estimators': 100,
    # Fraction of training data to be radom
    'subsample':0.9,
    # Fraction of prediction data to be random
    'colsample_bytree': 0.8,
    # Minimum loss reduction
    'gamma': 0,
    # L1 regularization
    'reg_alpha': 0,
    # L2 regularization
    'reg_lambda': 2
}

In [ ]:
# Number of training rounds
num_boost_round = 150

In [ ]:
# Train the model
bst = xgb.train(params, dtrain, num_boost_round)

In [ ]:
# Make predictions
xgb_predict = bst.predict(dtest)

In [ ]:
mse = mean_squared_error(y_test, xgb_predict)
print(f'MSE: {mse}')

rmse = mean_squared_error(y_test, xgb_predict, squared=False)
print(f'RMSE: {rmse}')

r2 = r2_score(y_test, xgb_predict)
print(f'R^2 Score: {r2}')

# LightGBM

In [ ]:
X = df_columns_drop[features]
y = df_columns_drop[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
lgb_model = lgb.LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)

lgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='l2')

In [ ]:
y_pred = lgb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)

# Perform predictions

In [ ]:
user_input = {
    'above_grade_square_feet': float(input("Enter the area of your livable space (exclude basement): ")),
    'total_square_feet': float(input("Enter your house total area: ")),
    'garages': int(input("How many garages does your house have: ")),
    'beds': int(input("How many beds does your house have: ")),
    'baths': int(input("How many baths does your house have: ")),
    'zip': int(input("What is the zip code of your house: ")),
    'hoa_fee': float(input("Enter your HOA fee (0 if none): ")),
    'age_of_building': float(input("Enter the age of your building: ")),
    'days_on_market': int(input("Enter average days on market (0 if unknown): ")),
    'property_type_Detached Single': input("Is your property type Detached Single? (yes/no): "),
    'property_type_Mobile Home': input("Is your property type Mobile Home? (yes/no): "),
    'property_type_Multi-family': input("Is your property type Multi-family? (yes/no): "),
    'property_type_Two to Four Units': input("Is your property type Two to Four Units? (yes/no): "),
    'structural_type_Condo': input("Is your building structural type Condo? (yes/no): "),
    'structural_type_Detached': input("Is your building structural type Detached? (yes/no): "),
}

In [ ]:
user_df = pd.DataFrame([user_input])

mapping_dictionary = {'no': 0, 'yes': 1}

property_columns = [
    'property_type_Detached Single',
    'property_type_Mobile Home',
    'property_type_Multi-family',
    'property_type_Two to Four Units'
]

structural_columns = [
    'structural_type_Condo',
    'structural_type_Detached'
]

for column in property_columns + structural_columns:
    user_df[column] = user_df[column].map(mapping_dictionary)

In [ ]:
user_prediction = lgb_model.predict(user_df)

print(f"The predicted sold price for your house is: {user_prediction[0]:,.2f} USD")

# Data Clustering

In [ ]:
# Copy the dataset for clustering
df_clustering = df_columns_drop.copy()

In [ ]:
# Reselect the feature and target columns
clustering_df = df_clustering[[
    'above_grade_square_feet',
    'total_square_feet',
    'garages',
    'beds',
    'baths',
    'zip',
    'hoa_fee',
    'age_of_building',
    'days_on_market',
    'property_type_Detached Single',
    'property_type_Mobile Home',
    'property_type_Multi-family',
    'property_type_Two to Four Units',
    'structural_type_Condo',
    'structural_type_Detached'
]]

In [ ]:
# Optimal number of cluster that can be formed
Elbow = KElbowVisualizer (KMeans(), k = 10)
Elbow.fit(clustering_df)
Elbow.show

In [ ]:
# Group the houses to their relevant cluster
num_clusters = 
kmeans = KMeans(n_clusters = num_clusters)
kmeans.fit(clustering_df)
clustering_df['Cluster'] = kmeans.labels_

In [ ]:
# Group 0
clustering_0 = clustering_df[clustering_df['Cluster'] == 0]
clustering_0.describe()

In [ ]:
# Group 1
clustering_1 = clustering_df[clustering_df['Cluster'] == 1]
clustering_1.describe()

In [ ]:
# Group 2
clustering_2 = clustering_df[clustering_df['Cluster'] == 2]
clustering_2.describe()

In [ ]:
# Group 3
clustering_3 = clustering_df[clustering_df['Cluster'] == 3]
clustering_3.describe()

In [ ]:
# Group 4
clustering_4 = clustering_df[clustering_df['Cluster'] == 4]
clustering_4.describe()

In [ ]:
# Group 5
clustering_5 = clustering_df[clustering_df['Cluster'] == 5]
clustering_5.describe()

# Data Visualization

In [ ]:
# For visualization
df_visualization = df_columns_drop.copy()

In [ ]:
# Heatmap
numeric_df = df_columns_drop[features + target]
corr_matrix = numeric_df.corr()

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    width=1200,
    height=1200
)
fig.show()

In [ ]:
# Scatter plots
scatter_features = [
    'above_grade_square_feet',
    'total_square_feet',
    'beds',
    'baths',
    'garages',
    'hoa_fee',
    'age_of_building',
    'days_on_market'
]

fig = make_subplots(
    rows=2,
    cols=4,
    subplot_titles=scatter_features
)

row = 1
col = 1

for feature in scatter_features:
    fig.add_trace(
        px.scatter(
            df_columns_drop,
            x=feature,
            y='sold_price'
        ).data[0],
        row=row,
        col=col
    )

    col += 1
    if col > 4:
        col = 1
        row += 1

fig.update_layout(
    height=800,
    width=1600,
    title_text="Key Features vs Sold Price"
)

fig.show()

In [ ]:
# Box plots
cat_features = [
    'property_type_Detached Single',
    'property_type_Mobile Home',
    'property_type_Multi-family',
    'property_type_Two to Four Units',
    'structural_type_Condo',
    'structural_type_Detached'
]

fig = make_subplots(rows=2, cols=3, subplot_titles=cat_features)

row = 1
col = 1
for feature in cat_features:
    fig.add_trace(
        px.box(df_columns_drop, x=feature, y='sold_price').data[0],
        row=row, col=col
    )
    col += 1
    if col > 3:
        col = 1
        row += 1

fig.update_layout(height=800, width=1600, title_text="Sold Price Distribution by Categorical Features")
fig.show()

# Data Dictionary

## Target
| Column | Description | Type |
|--------|-------------|------|
| sold_price | Final selling price of the property in USD | float |

## Numeric Features
| Column | Description | Type |
|--------|-------------|------|
| above_grade_square_feet | Square footage of the livable area excluding basement | float |
| total_square_feet | Total square footage of the property | float |
| garages | Number of garages in the property | float |
| beds | Number of bedrooms | float |
| baths | Number of bathrooms | float |
| zip | ZIP code of the property location | float |
| hoa_fee | Monthly Homeowner Association fee in USD | float |
| age_of_building | Age of the building in years | float |
| days_on_market | Number of days the property was listed before being sold | int |

## Categorical / Boolean Features
| Column | Description | Type |
|--------|-------------|------|
| property_type_Detached Single | Indicates if property type is Detached Single | bool |
| property_type_Mobile Home | Indicates if property type is Mobile Home | bool |
| property_type_Multi-family | Indicates if property type is Multi-family | bool |
| property_type_Two to Four Units | Indicates if property type is Two to Four Units | bool |
| structural_type_Condo | Indicates if the building is a Condo | bool |
| structural_type_Detached | Indicates if the building is Detached | bool |